In [92]:
import requests
import json
import facebook
import pandas as pd
from datetime import datetime
from datetime import date

class FeedCrawler():
    def __init__(self, token, page_id):
        self.token = token
        self.page_id = page_id
        self.graph = facebook.GraphAPI(access_token = self.token, version="2.12") #套件會用你的token連接到facebook
    def get_result(self):
        result = requests.get('https://graph.facebook.com/v2.9/{}/posts?&access_token={}'.format(self.page_id,self.token))
        return json.dumps(result.json(), indent=2)
    def get_posts_info(self,year, month, date, connection_name = 'posts'):

        time_range = datetime(year, month, date)
        posts = self.graph.get_all_connections(id=self.page_id,
                                       connection_name=connection_name,                                    
                                       since=time_range)
        post_ids =[]
        for ind, post in enumerate(posts):
#             print(ind, post['id'])
            post_ids.append(post['id'])
            
        df =  pd.DataFrame(columns = ['id','created_time','message','post_impressions','post_engaged_users','reply','reactions','interaction','link'])

        for i in post_ids:
            post = self.graph.get_object(id=i, fields='permalink_url,\
                                         comments.limit(0).summary(total_count),\
                                         message,reactions.limit(0).summary(total_count),\
                                         created_time,\
                                         insights.metric(post_impressions,post_engaged_users)')

            if 'message' in post:
                post_info = post['message']
            else:
                post_info = None
            time_stamp = post['created_time']
            post_id = post['id']
            post_impressions = post['insights']['data'][0]['values'][0]['value']
            post_engaged_users = post['insights']['data'][1]['values'][0]['value']
            reply = post['comments']['summary']['total_count']
            reaction = post['reactions']['summary']['total_count']
            interaction = reply + reaction
            link = post['permalink_url']

            test_df = pd.DataFrame(data = {
                'id':post_id,
                'created_time':time_stamp,
                'post_impressions':post_impressions,
                'post_engaged_users':post_engaged_users,
                'message':post_info,
                'reply':reply,
                'reactions':reaction,
                'interaction':interaction,
                'link':link},index=[0])
            df = df.append(test_df, ignore_index=True)
        return df
    def get_comments_info(self,year, month, date, connection_name = 'posts'):
        time_range = datetime(year, month, date)
        posts = self.graph.get_all_connections(id=self.page_id,
                                       connection_name=connection_name,                                    
                                       since=time_range)
        post_ids =[]
        for ind, post in enumerate(posts):
#             print(ind, post['id'])
            post_ids.append(post['id'])
    
    
        df =  pd.DataFrame(columns = ['topics','created_time','message', 'id'])

        for i in post_ids:
            post = self.graph.get_object(id=i, fields='message')
            first_index = post['message'].find('【')
            final_index = post['message'].find('】')
            topics = post['message'][first_index:final_index+1] 

            comments = self.graph.get_connections(id=i, connection_name='comments',limit=1000)
            test_df = pd.DataFrame(comments['data'])
            test_df['topics'] = topics
            df = df.append(test_df, ignore_index=True)
            
        return df
def find_word(df, string):
    s_reply = 0
    s_reaction = 0
    count = 0
    for i in range(len(df)):
        if string in df['message'][i]:
            s_reply = s_reply + df['reply'][i]
            s_reaction = s_reaction + df['reactions'][i]
            count = count + 1
    print('{} count is : {}\n{} reply is : {}\n{} reaction is {}'.format(string,count,string, s_reply,string, s_reaction))